<a href="https://colab.research.google.com/github/SomdeepAcharyya/Fake-News-classifiaction/blob/main/TwitterRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import networkx as nx
from google.colab import files
import io
import pandas as pd
import spacy
from textblob import TextBlob
from spacy.lang.en import English
nlp = English()
import csv
from itertools import chain
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
spc = spacy.load('en_core_web_sm')
import http.client
import urllib.parse
from urllib.request import urlopen
import math
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import datetime
from time import strptime

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import pandas as pd
import re
import scipy.stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import networkx as nx
import time, sys
from IPython.display import clear_output
import ipywidgets as widgets

class TwitterRank:

  def update_progress(progress):
      bar_length = 80
      if isinstance(progress, int):
          progress = float(progress)
      if not isinstance(progress, float):
          progress = 0
      if progress < 0:
          progress = 0
      if progress >= 1:
          progress = 1

      block = int(round(bar_length * progress))
      

      text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
      out = widgets.Output()
      out.append_stdout(text)


  def create_vocab_list(df_in=None):
      """
      Get the glossary
      :return:List, each element is a word
      """
      for index, row in df_in.iterrows():
          vocab_list.extend(re.findall('\w+',row['Clean Tweet']))

      return list(set(vocab_list))


  def get_sim(t, i, j, row_normalized_dt):
      '''
      Get sim (i, j)
      '''
      return 1.0 - abs(row_normalized_dt[i,t] - row_normalized_dt[j,t])

  def get_Pt(t, tweets_list, friends_tweets_list, row_normalized_dt, relationship):
      '''
      Get Pt, Pt [i] [j] is the probability that i follows j and i is affected by j under topic t
      '''
      print('Creating transition probability for topic {}'.format(t))
      Pt = scipy.sparse.lil_matrix((relationship.get_shape()))
      #rows,cols = relationship.nonzero()
      #for row,col in zip(rows,cols):
      #    Pt[row,col] = tweets_list[col]/friends_tweets_list[row] * get_sim(t, row, col, row_normalized_dt)
      rel_c = relationship.tocoo()    
      for i,j in zip(rel_c.row, rel_c.col):
          Pt[i,j] = tweets_list[j]/friends_tweets_list[i] * get_sim(t, i, j, row_normalized_dt)
      return Pt


  def get_TRt(gamma, topic_number, Pt, Et, iter=1000, tolerance=1e-16):
      '''
      Get TRt, the influence matrix of each user under t topic
      :param gamma: Get tuning parameters in TRt's formula
      :param Pt: Pt Matrix, Pt [i] [j] represents the probability that i follows j, and i is affected by j under topic t
      :param Et: Et Matrix, Et [i] represents user i's attention to topic t, has been normalized, all elements are added to 1
      :param iter: Maximum number of iterations
      :param tolerance: Stop iteration after TRt iteration when Euclidean distance from iteration is less than tolerance
      :return: TRt,TRt[i]Represents the influence of user i under topic t
      '''
      TRt = Et[:,topic_number]
      old_TRt = TRt
      i = 0
      print('Calculating influence scores for users under topic {}'.format(topic_number))
      while i < iter:
          TRt = gamma * (Pt*TRt) + (1 - gamma) * Et[:,topic_number]
          euclidean_dis = np.linalg.norm(TRt - old_TRt)
          if euclidean_dis < tolerance:
              break
          old_TRt = TRt
          i += 1
      return TRt


  def get_doc_list(df_in):
      """
      Get a list, each element is a piece of document
      :param samples: Number of documents
      :return: np.array,Each element is a document
      """
      print('Collecting grouped tweets by user')
      doc_list = df_in.groupby('screen_name')['Clean Tweet'].apply(lambda x:' '.join(x)).values
      return doc_list

  def get_feature_matrix(doc_list):
      """
      Get the feature matrix of each document, each word as a feature
      :param doc_list: list,Each element is a document
      :return: i row and j column list, i is the number of samples, j is the number of features, and feature_matrix_ij represents the number of times that feature j appears in the i-th sample
      """
      print('Creating term-screen_name matrix')
      vectorizer = CountVectorizer()
      feature_matrix = vectorizer.fit_transform(raw_documents=doc_list)
      return feature_matrix, vectorizer


  def get_num_tweets_list(nx_graph,df_in):
      """
      Get the number of tweets per user
      :return: list,The i element is the number of tweets from the i user
      """
      print('Gathering tweet count for all users')
      screen_name = df_in['screen_name'].value_counts()
      num_nodes = len(nx_graph)
      num_tweets_list = np.zeros(shape=(num_nodes))
      for ind,node in enumerate(nx_graph):
          num_tweets_list[ind] = screen_name[node] 
          #update_progress(ind/num_nodes)
      return num_tweets_list

  def get_relationship(nx_graph):
      """
      Get user relationship matrix
      :param samples: Number of Users
      :return: i row and j column, relationship [i] [j] = 1 means j follows i
      """
      print('Creating relationship matrix')
      return nx.to_scipy_sparse_matrix(nx_graph)


  def get_friends_tweets_list(relationship, tweets_list):
      """
      Get the sum of the number of tweets that each user has followed
      :param relationship: User relationship matrix, i rows and j columns, relationship [i] [j] = 1 means j follows i
      :param tweets_list: list,The i element is the number of tweets from the i user
      :return: list,The i element is the sum of the tweets from everyone i followed
      """
      print('Gathering tweet counts for friends of each user')
      friends_tweets_list = np.zeros(shape=(relationship.get_shape()[0]))
      for i in range(relationship.get_shape()[0]):
          friends_tweets_list[i] = tweets_list[relationship[i].nonzero()[1]].sum()
          update_progress(i / relationship.get_shape()[0])
      return friends_tweets_list

  def get_top_topic_influencers(TR, nx_graph, num_topics=5, num_influencers=10):
      top_influencer_list = pd.DataFrame()
      for i, TRt in enumerate(TR):
          top_influencer_list['Topic' + str(i) + 'Influncers'] = pd.Series(TRt,index=nx_graph.nodes()).sort_values(ascending=False).head(num_influencers).index
      return top_influencer_list

  def get_TR(num_topics, tweets_list, friends_tweets_list, row_normalized_dt, col_normalized_dt, relationship,
            gamma=0.2, tolerance=1e-16):
      """
      Get a TR matrix that represents the influence of each user on each topic
      :param topics: Number of topics
      :param samples: User number
      :param tweets_list: list,The i element is the number of tweets from the i user
      :param friends_tweets_list: list,The i element is the sum of the tweets from everyone i followed
      :param row_normalized_dt: dt Row normalization matrix
      :param col_normalized_dt: dt Column normalization matrix
      :param relationship: i row and j column, relationship [i] [j] = 1 means j follows i
      :param gamma: Get the tuning parameters in the formula for TRt
      :param tolerance: Stop iteration after TRt iteration when Euclidean distance from iteration is less than tolerance
      :return: list,TR[i][j]Is the influence of user j on topic i
      """
      print('Ranking users by ')
      TR = np.zeros(shape=(num_topics,tweets_list.shape[0]))
      for i in range(num_topics):
          print('topic number {}'.format(i))
          Pt = get_Pt(i, tweets_list, friends_tweets_list, row_normalized_dt, relationship)
          Et = col_normalized_dt
          TR[i] = get_TRt(gamma, i, Pt, Et, tolerance).flatten()
      return TR

  def get_graph_object(df_in,source='screen_name',target='Retweet of',filter_column=None):
      """
      Get the network in the form of a networkx graph object
      :param df_in: The raw dataframe with screen_name and tweets
      return: network of screen_name as a networkx object with retweets as edges.
      """
      print('Creating graph object')
      G = nx.convert_matrix.from_pandas_edgelist(df_in,source,target)
      print('Removing nodes where screen_name doesn\'t exist in raw df')
      G.remove_nodes_from(list(df_in['Retweet of'][~df_in['Retweet of'].isin(df_in['screen_name'].value_counts().index)].unique()))
      return G

  def get_lda_model(topics, n_iter, df_in):
      """
      Get the trained LDA model
      :param topics: Number of topics
      :param n_iter: Number of iterations
      :return: model,LDA model after training
              vocab_list,A list of all words that have appeared in these documents, each element is a word
      """
      doc_list = TwitterRank.get_doc_list(df_in)
      #vocab_list = create_vocab_list(df_in)
      term_frequency, vectorizer = TwitterRank.get_feature_matrix(doc_list)
      #feature_matrix = term_frequency.toarray()
      vocab_list = vectorizer.get_feature_names()
      print('Fitting LDA model to discover topics')
      model = LatentDirichletAllocation(n_components=topics,max_iter=n_iter)
      model.fit(term_frequency)
      return model, vocab_list, term_frequency


  def print_topics_as_df(model, vocab_list, n_top_words=5):

      topic_word_df = pd.DataFrame(model.components_,columns=vocab_list)
      sorted_topic_words = pd.DataFrame()
      for index, row in topic_word_df.iterrows():
          row_df = pd.DataFrame({'topic_'+str(index): row.sort_values(ascending=False).index[:5].values})
          sorted_topic_words = pd.concat([sorted_topic_words,row_df],axis=1)
      return sorted_topic_words


  def get_TR_using_DT(dt, df_in, num_topics=5, gamma=0.2, tolerance=1e-16):
      """
      Knowing the DT matrix gives the TR matrix
      :param dt: dt The matrix represents the topic distribution of the document, and dt [i] [j] represents the proportion of the topic j in the document i
      :param samples: Number of documents
      :param topics:  Number of topics
      :param gamma: Get the tuning parameters in the formula for TRt
      :param tolerance: Stop iteration after TRt iteration when Euclidean distance from iteration is less than tolerance
      :return TR: matrix,TR[i][j]Is the influence of user j on topic i
      :return nx_graph: A Networkx graph object where the nodes are screen_name and edges are retweets
      """
      row_normalized_dt = dt/(dt.sum(axis=1).reshape(-1,1))
      col_normalized_dt = dt/dt.sum(axis=0)
      nx_graph = TwitterRank.get_graph_object(df_in,filter_column='Retweet of')
      relationship = TwitterRank.get_relationship(nx_graph)
      tweets_list = TwitterRank.get_num_tweets_list(nx_graph,df_in)
      friends_tweets_list = TwitterRank.get_friends_tweets_list(relationship, tweets_list)
      TR = TwitterRank.get_TR(num_topics, tweets_list, friends_tweets_list, row_normalized_dt, col_normalized_dt, relationship,
                  gamma, tolerance)
      return TR, nx_graph

  def twitter_rank(raw_df, topics=5, n_iter=100, gamma=0.2, tolerance=1e-16):
      """
      Twitter rank of documents
      :param topics: number of topics to discover
      :param n_iter: max number of iterations for LDA model
      :param gamma: The tuning parameters in the formula for TRt
      :param tolerance: Tolerance for early stopping
      :return:TR: A matrix of screen_name influence scores for each topic. Rows are screen_name, columns are topics
      :return:graph object: A Networkx graph object where the nodes are screen_name and edges are retweets 
      """
      model, vocab_list, term_frequency = get_lda_model(topics, n_iter, raw_df)
      #dt matrix represents the topic distribution of the document, 
      #dt [i] [j] represents the proportion of the subject j in the document i
      dt = model._unnormalized_transform(term_frequency)
      TR, graph_object = get_TR_using_DT(dt, raw_df, topics, gamma, tolerance)

      return TR, graph_object, model, vocab_list

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving UserTweet_0510.csv to UserTweet_0510.csv


In [ ]:
tweets_df = pd.read_csv('UserTweet_0510.csv')
tweets_df = tweets_df.dropna()
tweets_df = tweets_df.reset_index(drop=True)

In [ ]:
tweets_df

In [ ]:
def remove_links(tweet):
    tweet = re.sub('http\S+','',tweet)
    tweet = re.sub('(?!http://)bit.ly/\S+','',tweet)
    tweet = re.sub('[A-Za-z0-9.@]+\.[A-Za-z0-9]+/[A-Za-z0-9.@]+','',tweet)
    tweet = tweet.strip('[link]')
    return tweet
def extract_links(tweet):
    link1 = re.findall('http\S+',tweet)
    return list(set(link1.extend(re.findall('(?!http://)bit.ly\S+',tweet))))
def remove_users(tweet):
    tweet = re.sub('@[A-Za-z]+[A-Za-z0-9-_]','',tweet)
    return tweet
def remove_hashtags(tweet):
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)','', tweet)
    return tweet

In [ ]:
import nltk 
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words.add('rt')
nlp.vocab['rt'].is_stop = True
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in nlp(tweet)
                            if not nlp.vocab[word.text].is_stop] # remove stopwords

    tweet_token_list = [word.lemma_ if '#' not in word.text else word.text
                        for word in tweet_token_list] # apply lemmatization
    tweet = ' '.join(tweet_token_list)
    return tweet

In [ ]:
tweets_df['Clean Tweet'] = tweets_df['text'].apply(clean_tweet)

In [ ]:
def find_retweet_of(tweet):
    tweet = re.findall('(?<=http://twitter.com/)\w+',tweet)
    return tweet
tweets_df['Retweet of'] = tweets_df['text'].apply(lambda x:find_retweet_of(x) if not pd.isna(x) else x)
tweets_df['Retweet of']

0        []
1        []
2        []
3        []
4        []
         ..
36270    []
36271    []
36272    []
36273    []
36274    []
Name: Retweet of, Length: 36275, dtype: object

In [ ]:
def model_topics(algorithm, vocab, model_in=None):
    if model_in is None:
        if algorithm == 'LDA':
            model = LatentDirichletAllocation(n_components=10)
        elif algorithm == 'NMF':
            model = NMF(n_components=10)
        model.fit(tf)
        model.get_params()
    else: model = model_in
    topic_word_df = pd.DataFrame(model.components_,columns=vocab)
    sorted_topic_words = pd.DataFrame()
    for index, row in topic_word_df.iterrows():
        row_df = pd.DataFrame({'topic_'+str(index): row.sort_values(ascending=False).index[:5].values})
        sorted_topic_words = pd.concat([sorted_topic_words,row_df],axis=1)
    return topic_word_df, sorted_topic_words

In [ ]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 80
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
tweets_df['Clean Tweet'] = tweets_df['Clean Tweet'].apply(lambda x:str(x))

In [ ]:
tweets_df['Clean Tweet']

0                                                   butter
1          schedule update   amp happy camp   sbs inkig...
2        bts continue melt heart ahead new english lang...
3                                   # group teaser photo  
4                                        jenna andrews twt
                               ...                        
36270                                                     
36271                                                 vlog
36272                                                 vlog
36273                                                     
36274                                                     
Name: Clean Tweet, Length: 36275, dtype: object

In [ ]:
model_, vocab_, tf_ = TwitterRank.get_lda_model(5,10,tweets_df)
dt = model_._unnormalized_transform(tf_)
dt

Creating term-screen_name matrix
Fitting LDA model to discover topics


array([[2.02738559e-01, 8.85187750e+02, 2.04024235e-01, 2.00834904e-01,
        2.04652335e-01],
       [2.02004519e-01, 4.19361178e+00, 2.01134746e-01, 2.01185780e-01,
        2.02063175e-01],
       [2.00000000e-01, 2.00000000e-01, 2.00000000e-01, 2.00000000e-01,
        2.00000000e-01],
       ...,
       [2.00466453e-01, 2.02769586e-01, 6.18241845e+00, 2.01175239e-01,
        6.21317027e+00],
       [2.00094839e-01, 2.00039663e-01, 2.00059483e-01, 2.00038067e-01,
        1.19976795e+00],
       [2.00000000e-01, 2.00000000e-01, 2.00000000e-01, 2.00000000e-01,
        2.00000000e-01]])

In [ ]:
TR, graph = TwitterRank.get_TR_using_DT(dt, tweets_df, 5, gamma=0.2, tolerance=1e-16)

Creating graph object


TypeError: ignored

In [ ]:
final_ranks = TwitterRank.get_top_topic_influencers(TR,graph)
final_ranks

In [ ]:
topics = TwitterRank.print_topics_as_df(model_,vocab_)
topics